In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import numpy as np
import os
import seaborn as sns
from scipy.stats import uniform, skew, kurtosis

In [ ]:
file_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF\CH\Macassa_clearance\Box14\AdaptiveZ_10mm\Core_000_Box_014_of_020_Part_1_of_4"

In [ ]:
coords_file = os.path.join(file_path, '.component_parameters.txt')
with open(coords_file) as file:
        lines = file.readlines()
            
        for line in lines:
            if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())

In [ ]:
point_cloud = []

if os.path.isdir(file_path):
    lidar_files = [fn for fn in os.listdir(
        file_path) if fn.endswith('.bpc')]
    if any(lidar_files):
        lidar_filename = file_path + os.sep + lidar_files[0]

data = np.fromfile(lidar_filename, dtype=np.float32)
point_cloud = data.reshape(-1, 3)  # to xyz

ff = ~np.isnan(point_cloud).any(axis=1)
point_cloud = point_cloud[ff, ...]

point_cloud[:, 1] = point_cloud[:, 1] - float(y_offset)

print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")

In [ ]:

matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
with open(matrix_file, 'r') as file:
    lines = file.readlines()

transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

num_points = point_cloud.shape[0]

homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
transformed_points = homogeneous_points @ transformation_matrix.T
point_cloud_transformed = transformed_points[:, :3]
point_cloud = point_cloud_transformed
point_cloud = point_cloud[
    (point_cloud[:,0] >= x_stop) & 
    (point_cloud[:,0] <= x_start) & 
    (point_cloud[:,1] >= 50) &
    (point_cloud[:,1] <= 255)
    ]


print(f"# of point {point_cloud.shape[0]}")

In [ ]:
def print_properties(point_cloud, point_1, point_2):
    z_vals = point_cloud[(point_cloud[:, 0] >= point_1[0]) & 
                         (point_cloud[:, 0] <= point_2[0]) & 
                         (point_cloud[:, 1] >= point_1[1]) & 
                         (point_cloud[:, 1] <= point_2[1])][:, 2]
 
    plt.figure(figsize=(10, 5))
    plt.hist(z_vals, bins = 1000)
    plt.xlabel('Z values')
    plt.ylabel('Count')
    plt.title('Histogram of Z values in selected region')

    values = []

    mean = np.mean(z_vals)
    variance =  np.var(z_vals)
    std = np.std(z_vals)
    skw = skew(z_vals)
    kurt = kurtosis(z_vals)

    values.append(variance)
    values.append(std)
    values.append(skw)
    values.append(kurt)

    print(values) 

    statistics_text = (
    f"Mean: {mean:.4f}, "
    f"Variance: {variance:.4f}, "
    f"Standard Deviation: {std:.4f}, "
    f"Skewness: {skw:.4f}, "
    f"Kurtosis: {kurt:.4f}")

    plt.gcf().text(
        0.5, 0.04, statistics_text, 
        fontsize=10, ha='center', va='bottom', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white')
    )
    plt.tight_layout(rect=[0, 0.05, 1, 1])  
    plt.show()

In [ ]:
%matplotlib qt

x = point_cloud[:, 0]
y = point_cloud[:, 1]
z = point_cloud[:, 2]


print(point_cloud.shape)

downsampled_indices = np.random.choice(point_cloud.shape[0], size=100000, replace=False)
x_downsampled = x[downsampled_indices]
y_downsampled = y[downsampled_indices]
z_downsampled = z[downsampled_indices]

fig, ax = plt.subplots(figsize=(10, 4.5))  
scatter = ax.scatter(x_downsampled, y_downsampled, c=z_downsampled, cmap='binary', s=1)
plt.colorbar(scatter, label='Z value')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Point Cloud Heat Map (Downsampled)')

plt.xlim(x_stop, x_start)

plt.ion()
plt.show()

def onselect(eclick, erelease):
    start = (eclick.xdata, eclick.ydata)
    end = (erelease.xdata, erelease.ydata)
    print_properties(point_cloud,start,end)
    

rect_selector = RectangleSelector(
    ax,
    onselect,
    interactive=True,
    drag_from_anywhere=True,
    button=[1], 
    minspanx=5,
    minspany=5,
    spancoords='data'
)

